In [2]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv("airline_reviews_clean.csv")
df.head()

,Name,Review Date,Airline,Verified,Reviews,Type of Traveller,Month Flown,Route,Class,Seat Comfort,Staff Service,Food & Beverages,Inflight Entertainment,Value For Money,Overall Rating,Recommended,stemmed_content
0,Alison Soetantyo,2024-03-01,Singapore Airlines,True,Flight was amazing. The crew onboard this fl...,Solo Leisure,December 2023,Jakarta to Singapore,Business Class,4,4,4,4,4,9,1,flight amaz crew onboard flight welcom gave go...
1,Robert Watson,2024-02-21,Singapore Airlines,True,Booking an emergency exit seat still meant h...,Solo Leisure,February 2024,Phuket to Singapore,Economy Class,5,3,4,4,1,3,0,book emerg exit seat still meant huge discomfo...
2,S Han,2024-02-20,Singapore Airlines,True,Excellent performance on all fronts. I would...,Family Leisure,February 2024,Siem Reap to Singapore,Economy Class,1,5,2,1,5,10,1,excel perform front would definit choos use ai...
3,D Laynes,2024-02-19,Singapore Airlines,True,Pretty comfortable flight considering I was f...,Solo Leisure,February 2024,Singapore to London Heathrow,Economy Class,5,5,5,5,5,10,1,pretti comfort flight consid fli economi class...
4,A Othman,2024-02-19,Singapore Airlines,True,The service was consistently good from start ...,Family Leisure,February 2024,Singapore to Phnom Penh,Economy Class,5,5,5,5,5,10,1,servic consist good start finish cabin crew sh...


In [22]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load DistilGPT-2 model and tokenizer
model_name = "distilgpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)


In [20]:
# Set padding token to avoid warning
tokenizer.pad_token = tokenizer.eos_token

# Example: Use the 'stemmed content' column for text generation
stemmed_review = df['Reviews'].iloc[0]  # First review example

# Encode the input and create attention mask
inputs = tokenizer.encode(stemmed_review, return_tensors="pt", padding=True)
attention_mask = inputs.ne(tokenizer.pad_token_id).long()

# Generate text with attention mask and pad_token_id set
outputs = model.generate(
    inputs, 
    attention_mask=attention_mask, 
    pad_token_id=tokenizer.eos_token_id, 
    max_new_tokens=20,  # Further reduced length
    num_return_sequences=1, 
    no_repeat_ngram_size=2,
    temperature=0.6,
    repetition_penalty=1.2,
    do_sample=True,
    top_p=0.9  # Nucleus sampling to ensure better focus
)


# Decode the generated text to make it human-readable
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Generated Text: \n", generated_text)


Generated Text: 
   Flight was amazing. The crew onboard this flight were very welcoming, and gave a good atmosphere. The crew serving my aisle goes by the initial “G”. She was very kind & helpful. Gave my mom a bday cake for a late celebration even though it was just a 1hr 45min flight. Seat is well sanitized, legroom is spacious. IFE onboard has many variety of shows, music, etc. Bathroom always kept clean by crew at all times. & Food was delicious, overall this flight is a 9/10 experience!
Posted in: September 14th 2017 | Last updated on August 21st 2017


In [33]:
import torch
def generate_response(prompt):
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    attention_mask = torch.ones(inputs.shape, dtype=torch.long)  # Ensure correct attention mask

    outputs = model.generate(
        inputs, 
        attention_mask=attention_mask,
        pad_token_id=tokenizer.eos_token_id, 
        max_length=100,  # Increased length
        num_return_sequences=1, 
        no_repeat_ngram_size=2,
        temperature=0.7,  # Adjusted for varied but coherent responses
        repetition_penalty=1.2,
        do_sample=True,
        top_p=0.9
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example chatbot interaction
prompt = "Based on recent passenger reviews, summarize the common feedback about the quality of food served on Emirates Airlines."
response = generate_response(prompt)
print("Bot:", response)


Bot: Based on recent passenger reviews, summarize the common feedback about the quality of food served on Emirates Airlines.
The most frequently cited reasons for this is that there are a lot of different types and sizes available from various countries around the world to meet our needs while being able be more environmentally conscious in terms which can result even further if we don't have enough resources or infrastructure (or other factors) at all as well – it's also important not only because passengers understand how they're consuming their meals but due
